
# 📡 RF Kalibratie + Positievisualisatie (3× Raspberry Pi)

Deze notebook combineert:
1. **Kalibratie en ruisbepaling** van drie Raspberry Pi's  
2. **Interactieve visualisatie** van de Pi-posities en de geschatte GSM-positie met logaritmisch afstandsmodel  

De parameters (C, n, σ) worden automatisch ingeladen uit `rssi_parameters.json`.  
Je kunt de posities van de Pi's en GSM met sliders wijzigen om de geometrie en cirkels te verkennen.


In [1]:

import json
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, fixed

# Inladen van eerder berekende parameters
with open("rssi_parameters_AndereMeetwaarden.json", "r") as f:
    params = json.load(f)

combined = params["combined"]
per_pi = params["individual"]

print("✅ Parameters geladen uit rssi_parameters_AndereMeetwaarden.json")

# We tonen ze mooi in een tabelvorm
import pandas as pd
df_params = pd.DataFrame(per_pi).T
display(df_params.round(4))

print("\nGecombineerd model:")
for k, v in combined.items():
    print(f"{k:25s}: {v:.4f}")


✅ Parameters geladen uit rssi_parameters_AndereMeetwaarden.json


,path_loss_n,C_intercept,slope_c1,sigma_residual_db,R2
Pi4,1.7924,-56.1096,-17.9238,4.5189,0.7728
Pi5 rood,2.2163,-51.7790,-22.1634,1.7902,0.9707
Pi5 zwart,1.8124,-57.3208,-18.1242,2.3381,0.9285



Gecombineerd model:
path_loss_n              : 1.9404
C_intercept              : -55.0698
slope_c1                 : -19.4038
sigma_residual_db        : 2.8824
R2                       : 0.8907


In [2]:

# Model: RSSI(d) = C - 10*n*log10(d)
def predicted_rssi(d, C, n):
    return C - 10 * n * np.log10(d)

# Omgekeerd: afstand uit RSSI
def distance_from_rssi(rssi, C, n):
    return 10 ** ((C - rssi) / (10 * n))


In [3]:
import numpy as np

def trilaterate(p1, p2, p3, r1, r2, r3):
    
    """
    Berekent (x, y) waar de drie cirkels met centra p1, p2, p3 en stralen r1, r2, r3 elkaar snijden.
    Geeft een benaderde oplossing via lineaire algebra (least squares).
    """

    # Omzetten naar numpy array
    P1, P2, P3 = np.array(p1), np.array(p2), np.array(p3)

    # Formules gebaseerd op lineaire verschillen van cirkelvergelijkingen
    A = 2 * np.array([
        [P2[0] - P1[0], P2[1] - P1[1]],
        [P3[0] - P1[0], P3[1] - P1[1]],
    ])
    b = np.array([
        r1**2 - r2**2 - np.dot(P1, P1) + np.dot(P2, P2),
        r1**2 - r3**2 - np.dot(P1, P1) + np.dot(P3, P3),
    ])

    try:
        sol = np.linalg.lstsq(A, b, rcond=None)[0]
        return sol
    except np.linalg.LinAlgError:
        return np.mean([P1, P2, P3], axis=0)


In [4]:
from ipywidgets import interact, FloatSlider
import matplotlib.pyplot as plt
import numpy as np


# Kleuren per Pi
colors = {
    "pi4": "gold",
    "pi5_zwart": "black",
    "pi5_rood": "red",
}

# mapping slider → JSON keys
name_map = {
    "pi4": "Pi4",
    "pi5_zwart": "Pi5 zwart",
    "pi5_rood": "Pi5 rood",
}

def plot_positions(pi4_x, pi4_y,
                   pi5_zwart_x, pi5_zwart_y,
                   pi5_rood_x, pi5_rood_y,
                   gsm_x, gsm_y,
                   n_factor):

    plt.figure(figsize=(7, 7))
    ax = plt.gca()

    pis = {
        "pi4":       (pi4_x,      pi4_y),
        "pi5_zwart": (pi5_zwart_x, pi5_zwart_y),
        "pi5_rood":  (pi5_rood_x,  pi5_rood_y),
    }

    gsm = (gsm_x, gsm_y)
    d_errs = []
    r_meas = {}

    # ---- Simuleer metingen ----
    for sim_name, (x, y) in pis.items():

        json_name = name_map[sim_name]

        C_i     = per_pi[json_name]["C_intercept"]
        n_i     = per_pi[json_name]["path_loss_n"] * n_factor
        sigma_i = per_pi[json_name]["sigma_residual_db"]

        # echte afstand
        d_true = np.hypot(gsm_x - x, gsm_y - y)

        # gemeten RSSI + ruis
        rssi_clean = C_i - 10 * n_i * np.log10(d_true)
        rssi_meas  = rssi_clean + np.random.normal(0.0, sigma_i)

        # afstand inschatten
        d_meas = 10 ** ((C_i - rssi_meas) / (10 * n_i))
        r_meas[sim_name] = d_meas

        # foutbalken
        d_upper = d_meas * 10 ** ( sigma_i / (10 * n_i))
        d_lower = d_meas * 10 ** (-sigma_i / (10 * n_i))
        d_err   = (d_upper - d_lower) / 2
        d_errs.append(d_err)

        # ---- TEKENEN ----
        color = colors[sim_name]

        # anker
        ax.scatter(x, y, color=color, s=100,
                   label=f"{sim_name} (n={n_i:.2f}, σ={sigma_i:.2f})")

        # grijze cirkels (ruisgebied)
        ax.add_patch(plt.Circle((x, y), d_meas,  color="gray", alpha=0.25))
        ax.add_patch(plt.Circle((x, y), d_upper, color="gray", alpha=0.1))
        ax.add_patch(plt.Circle((x, y), d_lower, color="gray", alpha=0.1))

        # naam erbij
        ax.text(x + 0.1, y + 0.1, sim_name, fontsize=9, color=color)

    # GSM true
    ax.scatter(gsm_x, gsm_y, color="red", marker="x", s=120, label="GSM true")

    # Trilateratie
    est_x, est_y = trilaterate(
        pis["pi4"], pis["pi5_zwart"], pis["pi5_rood"],
        r_meas["pi4"], r_meas["pi5_zwart"], r_meas["pi5_rood"]
    )

    est_r = np.mean(d_errs)

    # rode zone rond geschatte positie
    ax.add_patch(plt.Circle((est_x, est_y), est_r, color="red", alpha=0.15))
    ax.scatter(est_x, est_y, color="red", s=90, label="Estimated GSM")

    plt.axis("equal")
    plt.grid(True)
    plt.legend()
    plt.show()

    print(f"→ Estimated GSM: ({est_x:.2f}, {est_y:.2f})")
    print(f"→ Localization error: {np.hypot(est_x - gsm_x, est_y - gsm_y):.2f} m")


# Sliders
interact(
    plot_positions,
    pi4_x=FloatSlider(value=-1, min=-10, max=10, step=0.1),
    pi4_y=FloatSlider(value=0,  min=-10, max=10, step=0.1),
    pi5_zwart_x=FloatSlider(value=1,  min=-10, max=10, step=0.1),
    pi5_zwart_y=FloatSlider(value=0,  min=-10, max=10, step=0.1),
    pi5_rood_x=FloatSlider(value=0,  min=-10, max=10, step=0.1),
    pi5_rood_y=FloatSlider(value=2,  min=-10, max=10, step=0.1),
    gsm_x=FloatSlider(value=1,  min=-10, max=10, step=0.1),
    gsm_y=FloatSlider(value=1,  min=-10, max=10, step=0.1),
    n_factor=FloatSlider(value=1, min=0.1, max=2, step=0.05),
)


interactive(children=(FloatSlider(value=-1.0, description='pi4_x', max=10.0, min=-10.0), FloatSlider(value=0.0…

<function __main__.plot_positions(pi4_x, pi4_y, pi5_zwart_x, pi5_zwart_y, pi5_rood_x, pi5_rood_y, gsm_x, gsm_y, n_factor)>

De sigma afh van de afstand van de pi tov de gsm dus bv als pi1 1m van de gsm staat => die sigma gebruiken enzo => Als de sigma waarde van die afstand niet gekend is gebruik de last sigma waarde

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# ============================================================
# 1. JSON BESTANDEN LADEN
# ============================================================

# Sigma tabel laden
try:
    with open("sigma_parameters.json", "r") as f:
        sigma_table = json.load(f)
    print("✔ Sigma-tabel geladen")
except FileNotFoundError:
    print("❌ LET OP: 'sigma_parameters.json' niet gevonden.")
    sigma_table = {}

# RSSI parameters laden
try:
    with open("rssi_parameters_AndereMeetwaarden.json", "r") as f:
        params = json.load(f)
    per_pi = params["individual"]
    print("✔ RSSI parameters geladen")
except FileNotFoundError:
    print("❌ LET OP: 'rssi_parameters_AndereMeetwaarden.json' niet gevonden.")
    per_pi = {}

# ============================================================
# 2. HULPFUNCTIES (Sigma & Trilateratie)
# ============================================================

def get_sigma_for_distance(pi_name, d):
    """
    Haalt de sigma-waarde op van de Pi o.b.v. de afstand:
    - Kijkt in de lookup tabel (sigma_table)
    - Neemt de dichtstbijzijnde afstand (nearest neighbour)
    """
    key = pi_name.lower() 
    
    if key not in sigma_table:
        return 2.0 # Fallback als Pi niet in tabel staat

    available = np.array([float(x) for x in sigma_table[key].keys()])
    available_sorted = np.sort(available)

    # als groter dan max afstand in tabel -> gebruik laatste bekende sigma
    if d >= available_sorted[-1]:
        return float(sigma_table[key][str(available_sorted[-1])])

    # anders: nearest neighbour interpolation
    idx = np.abs(available_sorted - d).argmin()
    nearest = available_sorted[idx]
    return float(sigma_table[key][str(nearest)])

def trilaterate(p1, p2, p3, r1, r2, r3):
    """
    Berekent snijpunt van 3 cirkels (lineaire algebra methode).
    """
    (x1, y1), (x2, y2), (x3, y3) = p1, p2, p3

    A = np.array([
        [2*(x2-x1), 2*(y2-y1)],
        [2*(x3-x1), 2*(y3-y1)]
    ])

    b = np.array([
        r1**2 - r2**2 - x1**2 + x2**2 - y1**2 + y2**2,
        r1**2 - r3**2 - x1**2 + x3**2 - y1**2 + y3**2
    ])

    try:
        sol = np.linalg.solve(A, b)
        return sol[0], sol[1]
    except np.linalg.LinAlgError:
        return np.nan, np.nan

# ============================================================
# 3. MAPPINGS & KLEUREN
# ============================================================

name_map = {
    "pi4": "Pi4",
    "pi5_zwart": "Pi5 zwart",
    "pi5_rood": "Pi5 rood",
}

colors = {
    "pi4": "gold",
    "pi5_zwart": "black",
    "pi5_rood": "red",
}

# ============================================================
# 4. INTERACTIEVE SIMULATIEPLOT
# ============================================================

def plot_positions(pi4_x, pi4_y,
                   pi5_zwart_x, pi5_zwart_y,
                   pi5_rood_x, pi5_rood_y,
                   gsm_x, gsm_y,
                   n_factor): # <--- HIER KOMT DE SLIDER WAARDE BINNEN

    plt.figure(figsize=(9, 9))
    ax = plt.gca()

    pis = {
        "pi4":       (pi4_x, pi4_y),
        "pi5_zwart": (pi5_zwart_x, pi5_zwart_y),
        "pi5_rood":  (pi5_rood_x, pi5_rood_y),
    }

    r_meas = {}
    d_errs = []

    # ------------------------------------------------------
    # PER PI: SIMULATIE
    # ------------------------------------------------------
    for name, (x, y) in pis.items():
        json_name = name_map[name]

        # 1. Echte afstand berekenen
        d_true = np.hypot(gsm_x - x, gsm_y - y)

        # 2. Parameters ophalen
        if json_name in per_pi:
            C_i = per_pi[json_name]["C_intercept"]
            n_base = per_pi[json_name]["path_loss_n"]
        else:
            C_i, n_base = -50, 2.0 # Fallback
        
        # 3. TOEPASSEN N_FACTOR (De slider beïnvloedt n)
        n_final = n_base * n_factor

        # 4. Sigma ophalen (uit lookup table o.b.v. afstand)
        sigma_i = get_sigma_for_distance(name, d_true)

        # 5. RSSI genereren (Simulatie van meting)
        # We gebruiken n_final hier, want de omgeving verandert
        rssi_clean = C_i - 10 * n_final * np.log10(max(d_true, 0.001))
        rssi_meas  = rssi_clean + np.random.normal(0, sigma_i)

        # 6. Afstand terugrekenen (Het model)
        d_meas = 10 ** ((C_i - rssi_meas) / (10 * n_final))
        r_meas[name] = d_meas

        # 7. Onzekerheidszone berekenen
        # Let op: n_final staat in de noemer.
        # Als n klein is (slider laag), wordt de exponent groot -> Grote grijze cirkel
        d_upper = d_meas * 10 ** (sigma_i / (10 * n_final))
        d_lower = d_meas * 10 ** (-sigma_i / (10 * n_final))
        d_err = (d_upper - d_lower) / 2
        d_errs.append(d_err)

        # --- TEKENEN ---
        # Ankerpunt
        ax.scatter(x, y, s=120, color=colors[name], zorder=5,
                   label=f"{name}\n(n={n_final:.2f}, σ={sigma_i:.1f})")

        ax.text(x + 0.15, y + 0.15, name, color=colors[name], fontweight='bold')

        # Grijze onzekerheidsringen
        ax.add_patch(plt.Circle((x, y), d_meas, fill=False, edgecolor=colors[name], linestyle='--', alpha=0.6))
        ax.add_patch(plt.Circle((x, y), d_upper, color="gray", alpha=0.1))
        ax.add_patch(plt.Circle((x, y), d_lower, color="gray", alpha=0.1))

    # ------------------------------------------------------
    # TRUE GSM & TRILATERATIE
    # ------------------------------------------------------
    
    # Echte GSM positie
    ax.scatter(gsm_x, gsm_y, color="red", marker="x", s=180, linewidths=3, zorder=6, label="GSM True")

    # Trilateratie berekenen
    est_x, est_y = trilaterate(
        pis["pi4"], pis["pi5_zwart"], pis["pi5_rood"],
        r_meas["pi4"], r_meas["pi5_zwart"], r_meas["pi5_rood"]
    )

    # Gemiddelde foutstraal voor visualisatie
    est_r = np.mean(d_errs)

    # Rode geschatte zone
    if not np.isnan(est_x):
        ax.add_patch(plt.Circle((est_x, est_y), est_r, alpha=0.2, color="red"))
        ax.scatter(est_x, est_y, s=150, color="red", edgecolor="black", zorder=6, label="Estimated GSM")
        
        error_dist = np.hypot(est_x - gsm_x, est_y - gsm_y)
        title_text = f"Localization Error: {error_dist:.2f} m"
    else:
        title_text = "Geen oplossing gevonden (cirkels overlappen niet)"

    ax.set_title(title_text, fontsize=14)
    plt.axis("equal")
    plt.grid(True, linestyle='--', alpha=0.5)
    
    # Legenda netjes buiten plot of in hoek
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    
    plt.tight_layout()
    plt.show()

# ============================================================
# 5. SLIDERS CONFIGURATIE
# ============================================================

interact(
    plot_positions,
    pi4_x=FloatSlider(value=-1, min=-10, max=10, step=0.1, description='Pi4 X'),
    pi4_y=FloatSlider(value=0, min=-10, max=10, step=0.1, description='Pi4 Y'),

    pi5_zwart_x=FloatSlider(value=1, min=-10, max=10, step=0.1, description='Pi5 Zwart X'),
    pi5_zwart_y=FloatSlider(value=0, min=-10, max=10, step=0.1, description='Pi5 Zwart Y'),

    pi5_rood_x=FloatSlider(value=0, min=-10, max=10, step=0.1, description='Pi5 Rood X'),
    pi5_rood_y=FloatSlider(value=2, min=-10, max=10, step=0.1, description='Pi5 Rood Y'),

    gsm_x=FloatSlider(value=1, min=-10, max=10, step=0.1, description='GSM X'),
    gsm_y=FloatSlider(value=1, min=-10, max=10, step=0.1, description='GSM Y'),

    n_factor=FloatSlider(value=1.0, min=0.1, max=2.5, step=0.05, description='n-factor'),
)

✔ Sigma-tabel geladen
✔ RSSI parameters geladen


interactive(children=(FloatSlider(value=-1.0, description='Pi4 X', max=10.0, min=-10.0), FloatSlider(value=0.0…

<function __main__.plot_positions(pi4_x, pi4_y, pi5_zwart_x, pi5_zwart_y, pi5_rood_x, pi5_rood_y, gsm_x, gsm_y, n_factor)>